In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py
import plotly.graph_objs as go
from sklearn.cluster import KMeans
import warnings
import os
warnings.filterwarnings("ignore")
py.offline.init_notebook_mode(connected = True)

In [4]:
customer = pd.read_csv("SC_data.csv")
print(customer.shape)
customer.head()

(34544, 26)


,PNRLocatorID,TicketNum,CouponSeqNbr,ServiceStartCity,ServiceEndCity,PNRCreateDate,ServiceStartDate,PaxName,EncryptedName,GenderCode,...,BaseFareAmt,TotalDocAmt,UFlyRewardsNumber,UflyMemberStatus,CardHolder,BookedProduct,EnrollDate,MarketingFlightNbr,MarketingAirlineCode,StopoverCode
0,AADJKH,3.377510e+12,1,SFO,MSP,11/10/2014,12/8/2014,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,230.70,276.2,NaN,NaN,NaN,NaN,NaN,392,SY,NaN
1,AADJKH,3.377510e+12,2,MSP,SFO,11/10/2014,12/11/2014,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,230.70,276.2,NaN,NaN,NaN,NaN,NaN,395,SY,O
2,AAFCUE,3.377510e+12,2,MCO,MSP,12/1/2014,12/21/2014,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,308.75,0.0,NaN,NaN,NaN,DSNY,NaN,342,SY,O
3,AAFCUE,3.377510e+12,1,MSP,MCO,12/1/2014,12/17/2014,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,308.75,0.0,NaN,NaN,NaN,DSNY,NaN,341,SY,NaN
4,AAFDZS,3.377410e+12,2,LAX,MSP,10/9/2014,10/30/2014,NEUMJO,4E45554D414E44696420493F7C20676574207468697320...,M,...,139.54,178.2,196189803.0,Standard,False,NaN,39:04.0,422,SY,O


In [56]:
customer.dtypes

PNRLocatorID             object
TicketNum               float64
CouponSeqNbr              int64
ServiceStartCity         object
ServiceEndCity           object
PNRCreateDate            object
ServiceStartDate         object
PaxName                  object
EncryptedName            object
GenderCode               object
birthdateid             float64
Age                     float64
PostalCode               object
BkdClassOfService        object
TrvldClassOfService      object
BookingChannel           object
BaseFareAmt             float64
TotalDocAmt             float64
UFlyRewardsNumber       float64
UflyMemberStatus         object
CardHolder               object
BookedProduct            object
EnrollDate               object
MarketingFlightNbr        int64
MarketingAirlineCode     object
StopoverCode             object
dtype: object

In [6]:
customer.isnull().sum()

PNRLocatorID                0
TicketNum                   0
CouponSeqNbr                0
ServiceStartCity            0
ServiceEndCity              0
PNRCreateDate               0
ServiceStartDate            0
PaxName                     0
EncryptedName               0
GenderCode                164
birthdateid               164
Age                       164
PostalCode              27183
BkdClassOfService           0
TrvldClassOfService         0
BookingChannel              0
BaseFareAmt                 0
TotalDocAmt                 0
UFlyRewardsNumber       27153
UflyMemberStatus        27153
CardHolder              27153
BookedProduct           22379
EnrollDate              27153
MarketingFlightNbr          0
MarketingAirlineCode        0
StopoverCode            17307
dtype: int64

In [7]:
customer.describe()

,TicketNum,CouponSeqNbr,birthdateid,Age,BaseFareAmt,TotalDocAmt,UFlyRewardsNumber,MarketingFlightNbr
count,3.454400e+04,34544.000000,34380.000000,34380.000000,34544.000000,34544.000000,7.391000e+03,34544.000000
mean,3.374290e+12,1.459385,44856.944968,40.403287,287.018126,317.322380,2.042285e+08,365.547534
std,2.571218e+09,0.565820,6890.659919,18.855714,180.113575,214.492038,1.495937e+07,238.485626
min,3.372050e+12,1.000000,18139.000000,0.000000,0.000000,0.000000,1.000018e+08,100.000000
25%,3.372110e+12,1.000000,39591.000000,26.000000,174.880000,194.000000,2.007800e+08,251.000000
50%,3.372110e+12,1.000000,44814.000000,40.000000,273.870000,307.000000,2.029959e+08,351.000000
75%,3.377290e+12,2.000000,50025.000000,55.000000,368.720000,415.400000,2.103695e+08,471.000000
max,3.377560e+12,6.000000,59917.000000,114.000000,1744.000000,5275.000000,2.410847e+08,8877.000000


In [8]:
## only keep records with known gender, birthdate and age.
customer = customer.dropna(axis=0, subset=['GenderCode', 'birthdateid', 'Age'])

In [9]:
## assign 0 to rewards number if it's null
customer['UFlyRewardsNumber'].fillna(0, inplace = True)

In [10]:
## assign non-ufly to member status if it's null
customer['UflyMemberStatus'].fillna("non-ufly", inplace = True)

In [11]:
## Discard duplicate records
customer.drop_duplicates(subset = ["PNRLocatorID","CouponSeqNbr","PaxName","ServiceStartCity","ServiceEndCity","ServiceStartDate"], keep = False, inplace = True)

In [12]:
customer.shape

(31575, 26)

In [13]:
customer.groupby("BookingChannel").count()

,PNRLocatorID,TicketNum,CouponSeqNbr,ServiceStartCity,ServiceEndCity,PNRCreateDate,ServiceStartDate,PaxName,EncryptedName,GenderCode,...,BaseFareAmt,TotalDocAmt,UFlyRewardsNumber,UflyMemberStatus,CardHolder,BookedProduct,EnrollDate,MarketingFlightNbr,MarketingAirlineCode,StopoverCode
BookingChannel,,,,,,,,,,,,,,,,,,,,,
DFW,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,0,10,0,11,11,5
FCM,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,25,22,25,27,27,8
HRL,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,0
JFK,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,0,1,1,0
LAN,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,0,0,0,4,4,3
LAS,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,0
LAX,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,0,1,1,0
MKE,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0,2,0,2,2,1
MSP,36,36,36,36,36,36,36,36,36,36,...,36,36,36,36,0,17,0,36,36,8


In [14]:
# Replace odd one off booking channels with 'Other'
customer.loc[~customer['BookingChannel'].isin(['Outside Booking', 'Reservations Booking', 'SCA Website Booking', 'SY Vacation', 'Tour Operator Portal']), 'BookingChannel'] = 'Other'

In [15]:
customer.groupby("BookingChannel").count()

,PNRLocatorID,TicketNum,CouponSeqNbr,ServiceStartCity,ServiceEndCity,PNRCreateDate,ServiceStartDate,PaxName,EncryptedName,GenderCode,...,BaseFareAmt,TotalDocAmt,UFlyRewardsNumber,UflyMemberStatus,CardHolder,BookedProduct,EnrollDate,MarketingFlightNbr,MarketingAirlineCode,StopoverCode
BookingChannel,,,,,,,,,,,,,,,,,,,,,
Other,95,95,95,95,95,95,95,95,95,95,...,95,95,95,95,25,58,25,95,95,25
Outside Booking,14054,14054,14054,14054,14054,14054,14054,14054,14054,14054,...,14054,14054,14054,14054,1348,2972,1348,14054,14054,8472
Reservations Booking,1162,1162,1162,1162,1162,1162,1162,1162,1162,1162,...,1162,1162,1162,1162,349,324,349,1162,1162,418
SCA Website Booking,14211,14211,14211,14211,14211,14211,14211,14211,14211,14211,...,14211,14211,14211,14211,4872,5220,4872,14211,14211,5837
SY Vacation,948,948,948,948,948,948,948,948,948,948,...,948,948,948,948,207,769,207,948,948,473
Tour Operator Portal,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,...,1105,1105,1105,1105,50,1105,50,1105,1105,546


In [16]:
customer.groupby('MarketingAirlineCode').count()

,PNRLocatorID,TicketNum,CouponSeqNbr,ServiceStartCity,ServiceEndCity,PNRCreateDate,ServiceStartDate,PaxName,EncryptedName,GenderCode,...,BookingChannel,BaseFareAmt,TotalDocAmt,UFlyRewardsNumber,UflyMemberStatus,CardHolder,BookedProduct,EnrollDate,MarketingFlightNbr,StopoverCode
MarketingAirlineCode,,,,,,,,,,,,,,,,,,,,,
F9,24,24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,0,0,0,24,23
SY,31551,31551,31551,31551,31551,31551,31551,31551,31551,31551,...,31551,31551,31551,31551,31551,6851,10448,6851,31551,15748


In [17]:
# Only keep records that involve SunCountry airlines tickets, for which MarketingAirlineCode=="SY".
SY_customer = customer.loc[customer['MarketingAirlineCode'] == 'SY']

In [18]:
SY_customer.shape

(31551, 26)

In [19]:
group = SY_customer

In [20]:
group['min'] = group.groupby('PNRLocatorID')['CouponSeqNbr'].transform('min')
group

,PNRLocatorID,TicketNum,CouponSeqNbr,ServiceStartCity,ServiceEndCity,PNRCreateDate,ServiceStartDate,PaxName,EncryptedName,GenderCode,...,TotalDocAmt,UFlyRewardsNumber,UflyMemberStatus,CardHolder,BookedProduct,EnrollDate,MarketingFlightNbr,MarketingAirlineCode,StopoverCode,min
0,AADJKH,3.377510e+12,1,SFO,MSP,11/10/2014,12/8/2014,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,276.20,0.0,non-ufly,NaN,NaN,NaN,392,SY,NaN,1
1,AADJKH,3.377510e+12,2,MSP,SFO,11/10/2014,12/11/2014,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,276.20,0.0,non-ufly,NaN,NaN,NaN,395,SY,O,1
2,AAFCUE,3.377510e+12,2,MCO,MSP,12/1/2014,12/21/2014,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,0.00,0.0,non-ufly,NaN,DSNY,NaN,342,SY,O,1
3,AAFCUE,3.377510e+12,1,MSP,MCO,12/1/2014,12/17/2014,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,0.00,0.0,non-ufly,NaN,DSNY,NaN,341,SY,NaN,1
4,AAFDZS,3.377410e+12,2,LAX,MSP,10/9/2014,10/30/2014,NEUMJO,4E45554D414E44696420493F7C20676574207468697320...,M,...,178.20,196189803.0,Standard,False,NaN,39:04.0,422,SY,O,1
5,AAFDZS,3.377410e+12,1,MSP,LAX,10/9/2014,10/27/2014,NEUMJO,4E45554D414E44696420493F7C20676574207468697320...,M,...,178.20,196189803.0,Standard,False,NaN,39:04.0,421,SY,O,1
6,AAJWOU,3.372110e+12,1,SEA,MSP,5/4/2013,5/4/2013,GEDDER,474544444544696420493F7C2067657420746869732072...,M,...,0.00,0.0,non-ufly,NaN,SALCL,NaN,288,SY,NaN,1
7,AAMRWP,3.377470e+12,1,MSP,LAX,7/14/2014,7/25/2014,DUONUY,44554F4E4744696420493F7C2067657420746869732072...,F,...,110.00,0.0,non-ufly,NaN,NaN,NaN,421,SY,NaN,1
8,AAMSHL,3.372110e+12,1,MSP,MCO,11/5/2014,12/3/2014,PEREDA,504552455A52414D4952455344696420493F7C20676574...,F,...,420.20,102080090.0,Standard,False,SSWMIR,00:00.0,341,SY,NaN,1
9,AAMSHL,3.372110e+12,2,MCO,MSP,11/5/2014,12/7/2014,PEREDA,504552455A52414D4952455344696420493F7C20676574...,F,...,420.20,102080090.0,Standard,False,SSWMIR,00:00.0,342,SY,O,1


In [21]:
SY_customer = group[group['min'] == 1]

In [22]:
SY_customer.shape

(31448, 27)

In [23]:
# Create a unique customer ID by concatenating name, gender and birthday
SY_customer['uid'] = SY_customer['PaxName'].map(str) + '_' + SY_customer['GenderCode'].map(str) + '_' + SY_customer['birthdateid'].map(str)

In [24]:
SY_customer.head()

,PNRLocatorID,TicketNum,CouponSeqNbr,ServiceStartCity,ServiceEndCity,PNRCreateDate,ServiceStartDate,PaxName,EncryptedName,GenderCode,...,UFlyRewardsNumber,UflyMemberStatus,CardHolder,BookedProduct,EnrollDate,MarketingFlightNbr,MarketingAirlineCode,StopoverCode,min,uid
0,AADJKH,3.377510e+12,1,SFO,MSP,11/10/2014,12/8/2014,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,0.0,non-ufly,NaN,NaN,NaN,392,SY,NaN,1,SHORLE_F_39225.0
1,AADJKH,3.377510e+12,2,MSP,SFO,11/10/2014,12/11/2014,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,0.0,non-ufly,NaN,NaN,NaN,395,SY,O,1,SHORLE_F_39225.0
2,AAFCUE,3.377510e+12,2,MCO,MSP,12/1/2014,12/21/2014,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,0.0,non-ufly,NaN,DSNY,NaN,342,SY,O,1,SOEFEV_M_56218.0
3,AAFCUE,3.377510e+12,1,MSP,MCO,12/1/2014,12/17/2014,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,0.0,non-ufly,NaN,DSNY,NaN,341,SY,NaN,1,SOEFEV_M_56218.0
4,AAFDZS,3.377410e+12,2,LAX,MSP,10/9/2014,10/30/2014,NEUMJO,4E45554D414E44696420493F7C20676574207468697320...,M,...,196189803.0,Standard,False,NaN,39:04.0,422,SY,O,1,NEUMJO_M_46051.0


In [25]:
# creating a new categorical variable "age_group" with the following posisble values: 
# "0-17", "18-24", "25-34", "35-54", "55+" and "N/A"
bins = [0,18,25,35,55,np.inf]
labels = ["0-17", "18-24", "25-34", "35-54", "55+"]
SY_customer['age_group'] = pd.cut(SY_customer['Age'], bins=bins, labels=labels, right=False)

In [76]:
SY_customer['ServiceStartDate'] = pd.to_datetime(SY_customer['ServiceStartDate'])
SY_customer['PNRCreateDate'] = pd.to_datetime(SY_customer['PNRCreateDate'])

In [59]:
# For a given PNR, figure out true origin city (source of first leg) 
# Sorting within groups based on column "CouponSeqNbr":
SY_customer['true_origin']=SY_customer.sort_values(['PNRLocatorID', 'PaxName', 'CouponSeqNbr', 'ServiceStartDate']).groupby(['PNRLocatorID', 'PaxName'])['ServiceStartCity'].transform(lambda x: x.iloc[0])

In [62]:
# For a given PNR, figure out final destination (target of last leg)
SY_customer['final_destination']=SY_customer.sort_values(['PNRLocatorID', 'PaxName', 'CouponSeqNbr', 'ServiceStartDate']).groupby(['PNRLocatorID', 'PaxName'])['ServiceEndCity'].transform(lambda x: x.iloc[-1])

In [66]:
# Is the booking a round-trip or one-way?
SY_customer.loc[SY_customer['true_origin'] == SY_customer['final_destination'], 'roundtrip'] = True
SY_customer.loc[SY_customer['true_origin'] != SY_customer['final_destination'], 'roundtrip'] = False

In [68]:
# What is the size of the group?
SY_customer['group_size'] = SY_customer.groupby('PNRLocatorID')['uid'].transform('nunique')

In [70]:
# Create a binary indicator "group" corresponding to whether it was a group or single party traveling.
SY_customer.loc[SY_customer['group_size'] > 1, 'group'] = True
SY_customer.loc[SY_customer['group_size'] <= 1, 'group'] = False

In [74]:
import datetime

In [ ]:
# Figure out which calendar quarter the trip took place in.
bins = [0,4,7,10,13]
labels = ["Q1", "Q2", "Q3", "Q4"]
SY_customer['seasonality'] = pd.cut(SY_customer['ServiceStartDate'].dt.month, bins=bins, labels=labels, right=False)

In [77]:
# How many days in advance was the trip booked?
SY_customer['days_pre_booked'] = SY_customer['ServiceStartDate'] - SY_customer['PNRCreateDate']

In [78]:
SY_customer.head()

,PNRLocatorID,TicketNum,CouponSeqNbr,ServiceStartCity,ServiceEndCity,PNRCreateDate,ServiceStartDate,PaxName,EncryptedName,GenderCode,...,min,uid,age_group,true_origin,true_destination,final_destination,roundtrip,group_size,group,days_pre_booked
0,AADJKH,3.377510e+12,1,SFO,MSP,2014-11-10,2014-12-08,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,1,SHORLE_F_39225.0,55+,SFO,SFO,SFO,True,1,False,28 days
1,AADJKH,3.377510e+12,2,MSP,SFO,2014-11-10,2014-12-11,SHORLE,53484F52455944696420493F7C20676574207468697320...,F,...,1,SHORLE_F_39225.0,55+,SFO,SFO,SFO,True,1,False,31 days
2,AAFCUE,3.377510e+12,2,MCO,MSP,2014-12-01,2014-12-21,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,1,SOEFEV_M_56218.0,0-17,MSP,MSP,MSP,True,1,False,20 days
3,AAFCUE,3.377510e+12,1,MSP,MCO,2014-12-01,2014-12-17,SOEFEV,534F4546464B455244696420493F7C2067657420746869...,M,...,1,SOEFEV_M_56218.0,0-17,MSP,MSP,MSP,True,1,False,16 days
4,AAFDZS,3.377410e+12,2,LAX,MSP,2014-10-09,2014-10-30,NEUMJO,4E45554D414E44696420493F7C20676574207468697320...,M,...,1,NEUMJO_M_46051.0,35-54,MSP,MSP,MSP,True,1,False,21 days


In [80]:
SY_customer.to_csv("SY_customer.csv")